In [1]:
import pandas as pd 
pay_bill = pd.read_csv('../data/credit_record.csv')
pay_bill

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C
...,...,...,...
1048570,5150487,-25,C
1048571,5150487,-26,C
1048572,5150487,-27,C
1048573,5150487,-28,C


In [2]:
def get_unused_user(df):
    df = df.groupby('ID').apply(lambda x: x['STATUS'].isin(['X']).all()).reset_index()
    df = df[df[0] == True]
    df.rename(columns={0: 'credit_decision'}, inplace=True)
    df['credit_decision'] = 0
    return df
unused_df = get_unused_user(pay_bill)
unused_df

,ID,credit_decision
2,5001713,0
3,5001714,0
4,5001715,0
16,5001731,0
17,5001732,0
...,...,...
45947,5150409,0
45948,5150410,0
45965,5150460,0
45979,5150481,0


In [3]:
credit_decision = pd.DataFrame()
used_df = pay_bill[pay_bill['STATUS'] != 'X']
credit_decision['ID'] = used_df['ID'].unique()

def get_used_credit_decision(id):
    '''Determine for a given df, if the client is good, bad or unused status according to the credit usage.'''
    client_status = used_df[used_df['ID'] == id]['STATUS'].tolist()
    c_count = client_status.count('C')
    total_count = len(client_status)
    if c_count / total_count >= 0.7:
        return 1
    else:
        return -1

credit_decision['credit_decision'] = credit_decision['ID'].map(get_used_credit_decision)
credit_decision

,ID,credit_decision
0,5001711,-1
1,5001712,-1
2,5001717,-1
3,5001718,-1
4,5001719,1
...,...,...
41444,5150480,1
41445,5150482,-1
41446,5150484,-1
41447,5150485,-1


In [4]:
credit_decision['credit_decision'].value_counts()

credit_decision
-1    29402
 1    12047
Name: count, dtype: int64

In [5]:
final_decision = pd.concat([unused_df, credit_decision])
final_decision

,ID,credit_decision
2,5001713,0
3,5001714,0
4,5001715,0
16,5001731,0
17,5001732,0
...,...,...
41444,5150480,1
41445,5150482,-1
41446,5150484,-1
41447,5150485,-1


In [6]:
# final_decision.to_csv('data/credit_decision.csv', index=False)